In [ ]:
import undetected_chromedriver as uc

import re
import requests
from bs4 import BeautifulSoup

from urllib.parse import quote
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin

In [ ]:
def get_rz_values(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        scripts = soup.find_all('script')

        # Pattern to match the desired script lines
        pattern = re.compile(r"RZ\.(\w+) = '(.*?)'")

        # Dictionary to store the results
        rz_values = {}

        for script in scripts:
            if script.string:
                matches = pattern.finditer(script.string)
                for match in matches:
                    key = match.group(1)
                    value = match.group(2)
                    rz_values[key] = value

        return rz_values

    return None

In [ ]:
def extract_users(html_string):
    soup = BeautifulSoup(html_string, 'html.parser')

    # Find all input elements with type "checkbox"
    checkboxes = soup.find_all('input', {'type': 'checkbox'})

    checked_users = []
    unchecked_users = []

    for checkbox in checkboxes:
        user_email = checkbox['value']
        if user_email.endswith('*'):
            checked_users.append(user_email[:-1])  # Remove the trailing asterisk
        else:
            unchecked_users.append(user_email)

    return {'checked_users': checked_users, 'unchecked_users': unchecked_users}


In [ ]:
def getPagePermissions(page_url,webspace):
    page_info=get_rz_values(page_url)
    if page_info is None:
        print("ERROR WITH PAGE")
        return []
    permissions_url = f"https://cms3.revize.com/revize/util/permissions-editform.jsp?webspace={webspace}&page_key={page_info['page_key']}&parent_key={page_info['parent_key']}&adminpanel&set=webspace_page_permissions.page_key%3D{quote(page_info['page_key'])}"
    driver.get(permissions_url)
    return extract_users(driver.page_source)['checked_users']

In [ ]:
def get_links_recursive(url, base_url, visited, depth):
    if depth == 0:
        return
    if url in visited:
        return
    print("Scanning:", url)
    visited.append(url)

    try:
        response = requests.get(url)
        if response.status_code != 200:
            return
    except requests.exceptions.RequestException:
        return

    soup = BeautifulSoup(response.content, 'html.parser')

    for link in soup.find_all('a'):
        href = link.get('href')
        if href:
            parsed_url = urlparse(href)
            current_base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
            if not parsed_url.netloc or current_base_url == base_url:
                next_url = urljoin(base_url, href)
                next_url = next_url.split("#")[0].lower()
                if next_url.endswith('php'):
                    get_links_recursive(next_url, base_url, visited, depth - 1)

def crawl_subdomain(start_url, depth):
    parsed_start_url = urlparse(start_url)
    base_url = f"{parsed_start_url.scheme}://{parsed_start_url.netloc}{parsed_start_url.path.rsplit('/', 1)[0]}/"
    visited = list()
    get_links_recursive(start_url, base_url, visited, depth)
    return visited

In [ ]:
start_url = 'https://sh.ingham.org/'
urls = crawl_subdomain(start_url,depth=6)
len(urls)

In [ ]:
webspace ="inghamcounty"

In [ ]:
options = uc.ChromeOptions()
options.add_argument('--window-size=1024,768')

try:
    driver = uc.Chrome(options=options)
except TypeError as e:
    if str(e) == 'expected str, bytes or os.PathLike object, not NoneType':
        raise ValueError('Chrome installation not found')
    raise e
driver.get(f"https://cms3.revize.com/revize/security/index.jsp?webspace={webspace}&filename=/index.php")

In [ ]:
out = {}
#webspace ="inghamparks"
for url in urls:
    print(url)
    permissions = getPagePermissions(url,webspace)
    print(permissions)
    out[url] = permissions
    print()